In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from siuba import _, select, rename, mutate, group_by, arrange, filter, summarize, distinct, left_join

In [21]:
census = pd.read_csv('cleaned_data/census_demographics.csv')
health = pd.read_csv('cleaned_data/asthma_clean_file.csv')
wildfires = pd.read_csv('cleaned_data/CLEAN_CA_Fire_Incidents_GROUPED.csv')
air_quality = pd.read_csv('aoq_agg_data.csv')

In [22]:
census = (census
  >> mutate(
      White_Perc = _.TotalWhiteNotHispanic/_.TotalPopulation,
      Poverty_Perc = _.TotalBelowPovertyLine/_.TotalPopulation,
      Total_Uninsured = _.TotalEmployedNoInsurance + _.TotalUnemployedNoInsurance + _.TotalNotInLaborForceNoInsurance,
      Uninsured_Perc = _.Total_Uninsured/_.TotalPopulation
      )
  )

In [23]:
wildfires = wildfires.rename(columns = {'Counties.1':'Wildfire_Count'})

wildfires = (wildfires
  >> select(
      _.County ==  _.Counties, _.Year == _.ArchiveYear, _.Acres_Burned == _.AcresBurned, _.Duration, _.Wildfire_Count,
      )
  )

In [24]:
air_quality = (air_quality
  >> select(
      _.Year == _.Year_, _.County ==  _.county_, _.AQ_Min == _.aqi_min, _.AQ_Max == _.aqi_max, _.AQ_Mean == _.aqi_mean, _.AQ_Median == _.aqi_median,
      )
  )

In [77]:
df_merge = (health
  >> filter(_.Category == "Total Population", _.Age == "18 and Over")
  >> rename(County = "Geography")
  >> left_join(_, select(census, _.County, _.TotalPopulation, _.MedianIncome, _.White_Perc, _.Poverty_Perc, _.Uninsured_Perc), on = "County")
  >> left_join(_, wildfires, on = ["County","Year"])
  >> left_join(_, air_quality, on = ["County","Year"])
  >> select(_.County, _.Year, _.Total_Population == _.TotalPopulation, _.Median_Income == _.MedianIncome, _.White_Perc, _.Poverty_Perc, _.Uninsured_Perc,
            _.Asthma_Incidents == _.Count, _.Asthma_Rate == _.Rate, _.Acres_Burned, _.Duration, _.Wildfire_Count,
           _.AQ_Min, _.AQ_Max, _.AQ_Mean, _.AQ_Median)
  >> arrange(_.County, _.Year)
)

In [78]:
df_merge

,County,Year,Total_Population,Median_Income,White_Perc,Poverty_Perc,Uninsured_Perc,Asthma_Incidents,Asthma_Rate,Acres_Burned,Duration,Wildfire_Count,AQ_Min,AQ_Max,AQ_Mean,AQ_Median
224,Alameda,2011,1656754.0,43583.0,0.314137,0.097529,0.037430,6706,56.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,Alameda,2012,1656754.0,43583.0,0.314137,0.097529,0.037430,6965,58.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,Alameda,2013,1656754.0,43583.0,0.314137,0.097529,0.037430,6705,55.0,478.0,0 days,4.0,0.0,119.0,40.067273,38.0
56,Alameda,2014,1656754.0,43583.0,0.314137,0.097529,0.037430,6332,51.6,NaN,NaN,NaN,0.0,119.0,33.979899,31.0
0,Alameda,2015,1656754.0,43583.0,0.314137,0.097529,0.037430,6513,53.1,3340.0,5 days,5.0,0.0,124.0,36.424754,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,Yuba,2015,76360.0,27877.0,0.551925,0.151545,0.060071,215,39.5,188.0,6 days,2.0,NaN,NaN,NaN,NaN
336,Yuba,2016,76360.0,27877.0,0.551925,0.151545,0.060071,202,37.7,400.0,0 days,1.0,NaN,NaN,NaN,NaN
392,Yuba,2017,76360.0,27877.0,0.551925,0.151545,0.060071,245,45.1,10981.0,950 days,6.0,NaN,NaN,NaN,NaN
448,Yuba,2018,76360.0,27877.0,0.551925,0.151545,0.060071,227,40.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
df_merge['Duration'] = df_merge['Duration'].str.split(' ').str[0]
df_merge['Acres_Burned'] = df_merge['Acres_Burned'].fillna(0)
df_merge['Wildfire_Count'] = df_merge['Wildfire_Count'].fillna(0)
df_merge['Duration'] = df_merge['Duration'].fillna(0)

In [80]:
df_merge = df_merge.dropna(subset=['Total_Population'])

In [81]:
df_merge.to_csv('cleaned_data/complete_merged_table.csv', encoding = 'utf-8', index = False)